# 多模激發法之行銷回應預測建模案例

#### Prof. Ching-Shih Tsou (Ph.D.) (鄒慶士教授, Ph.D.) at the Institute of Information and Decision Sciences/Center for Applications of Data Science (CADS), National Taipei University of Business (國立臺北商業大學資訊與決策科學研究所教授兼資料科學應用研究中心主任); the Chinese Academy of R Software (CARS) (中華R軟體學會創會理事長) and the Data Science and Business Applications Association of Taiwan (DSBA) (臺灣資料科學與商業應用協會創會理事長)

> 案例背景

* 行銷活動可能回應的名單與回應機率。
* 客戶的狀態是動態變化的。
* 模型週期性更新。

In [1]:
# 導入庫
import time  # 導入自帶時間庫

import numpy as np  # numpy庫
import pandas as pd  # pandas庫
from sklearn.preprocessing import OneHotEncoder  # 導入OneHotEncoder庫
from sklearn.model_selection import StratifiedKFold, cross_val_score  # 導入交叉檢驗算法
from sklearn.feature_selection import SelectPercentile, f_classif  # 導入特征選擇方法庫
from sklearn.ensemble import AdaBoostClassifier  # 導入集成算法
from sklearn.pipeline import Pipeline  # 導入Pipeline庫
from sklearn.metrics import accuracy_score  # 準確率指標

> 函數定義

In [2]:
# 基本狀態查看
def set_summary(df):
    '''
    查看數據集的記錄數、維度數、前2條數據、描述性統計和數據類型
    :param df: 數據框
    :return: 無
    '''
    print ('Data Overview')
    print ('Records: {0}\tDimension{1}'.format(df.shape[0], (df.shape[1] - 1)))  # 打印數據集X形狀
    print ('-' * 30)
    print (df.head())  # 打印前2條數據
    print ('-' * 30)
    print ('Data DESC')
    print (df.describe())  # 打印數據基本描述性信息
    print ('Data Dtypes')
    print (df.dtypes)  # 打印數據類型
    print ('-' * 60)

In [3]:
# 缺失值檢視
def na_summary(df):
    '''
    查看數據集的缺失數據縱行、橫列記錄數
    :param df: 數據框
    :return: 無
    '''
    na_cols = df.isnull().any(axis=0)  # 每一縱行是否具有缺失值
    print ('NA Cols:')
    print (na_cols)  # 查看具有缺失值的縱行
    print ('-' * 30)
    print ('valid records for each Cols:')
    print (df.count())  # 查看每一縱行有效值（非NA）的記錄數
    print ('-' * 30)
    na_lines = df.isnull().any(axis=1)  # 查看每一橫列是否具有缺失值
    print ('Total number of NA lines is: {0}'.format(na_lines.sum()))  # 查看具有缺失值的橫列總記錄數
    print ('-' * 30)

In [4]:
# 類樣本均衡檢視
def label_summary(df):
    '''
    查看每個類的樣本量分布
    :param df: 數據框
    :return: 無
    '''
    print ('Labesl samples count:')
    print (df['value_level'].groupby(df['response']).count())  # 以response為分類匯總維度對value_level縱行計數統計
    print ('-' * 60)

In [5]:
# 數據預處理
# 變量類型轉換
def type_con(df):
    '''
    轉換目標縱行的數據為特定數據類型
    :param df: 數據框
    :return: 類型轉換後的數據框
    '''
    var_list = {'edu': 'int32',
                'user_level': 'int32',
                'industry': 'int32',
                'value_level': 'int32',
                'act_level': 'int32',
                'sex': 'int32',
                'region': 'int32'
                }  # 字典：定義要轉換的縱行及其數據類型
    for var, type in var_list.items():  # 循環讀出縱行名和對應的數據類型
        df[var] = df[var].astype(type)  # 數據類型轉換
    print ('Data Dtypes')
    print (df.dtypes)  # 打印數據類型
    print ('-' * 30)
    return df

In [6]:
# NA值替換
def na_replace(df):
    '''
    將數據集中的NA值使用自定義方法替換
    :param df: 數據框
    :return: NA值替換後的數據框
    '''
    na_rules = {'age': df['age'].mean(),
                'total_pageviews': df['total_pageviews'].mean(),
                'edu': df['edu'].median(),
                'edu_ages': df['edu_ages'].median(),
                'user_level': df['user_level'].median(),
                'industry': df['user_level'].median(),
                'act_level': df['act_level'].median(),
                'sex': df['sex'].median(),
                'red_money': df['red_money'].mean(),
                'region': df['region'].median()
                }  # 字典：定義各個縱行數據轉換方法
    df = df.fillna(na_rules)  # 使用指定方法填充缺失值
    print ('Check NA exists:')
    print (df.isnull().any().sum())  # 查找是否還有缺失值
    print ('-' * 30)
    return df

In [7]:
# 標籤編碼
def symbol_con(df, enc_object=None, train=True):
    '''
    將分類和順序變量轉換為二值化的標籤變量
    :param df: 數據框
    :param enc_transform: sklearn的標籤編碼對象，訓練階段設置默認值為None；預測階段使用從訓練階段獲得的轉換對象
    :param train: 是否為訓練階段的判斷狀態，訓練階段為True，預測階段為False
    :return: 標籤編碼後的數據框、標籤編碼對象（如果是訓練階段）
    '''
    convert_cols = ['edu', 'user_level', 'industry', 'value_level', 'act_level', 'sex', 'region']  # 選擇要做標籤編碼的縱行名
    df_con = df[convert_cols]  # 選擇要做標籤編碼的數據
    df_org = df[['age', 'total_pageviews', 'edu_ages', 'blue_money', 'red_money', 'work_hours']].values  # 設置不作標籤編碼的縱行
    if train == True:  # 如果處於訓練階段
        enc = OneHotEncoder()  # 建立標籤編碼模型對象
        enc.fit(df_con)  # 訓練模型
        df_con_new = enc.transform(df_con).toarray()  # 轉換數據並輸出為數組格式
        new_matrix = np.hstack((df_con_new, df_org))  # 將未轉換的數據與轉換後的數據合併
        return new_matrix, enc
    else:
        df_con_new = enc_object.transform(df_con).toarray()  # 使用訓練階段獲得的轉換對象轉換數據並輸出為數組格式
        new_matrix = np.hstack((df_con_new, df_org))  # 將未轉換的數據與轉換後的數據合併
        return new_matrix

In [8]:
# 獲得最佳模型參數
def get_best_model(X, y):
    '''
    結合交叉檢驗得到不同參數下的分類模型結果
    :param X: 輸入X（特征變量）
    :param y: 預測y（目標變量）
    :return: 特征選擇模型對象
    '''
    transform = SelectPercentile(f_classif, percentile=50)  # 使用f_classif方法選擇特征最明顯的50%數量的特征
    model_adaboost = AdaBoostClassifier()  # 建立AdaBoostClassifier模型對象
    model_pipe = Pipeline(steps=[('ANOVA', transform), ('model_adaboost', model_adaboost)])  # 建立由特征選擇和分類模型構成的“管道”對象
    cv = StratifiedKFold(5)  # 設置交叉檢驗次數
    n_estimators = [20, 50, 80, 100]  # 設置模型參數列表
    score_methods = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']  # 設置交叉檢驗指標
    mean_list = list()  # 建立空列表用於存放不同參數方法、交叉檢驗評估指標的均值列表
    std_list = list()  # 建立空列表用於存放不同參數方法、交叉檢驗評估指標的標準差列表
    for parameter in n_estimators:  # 循環讀出每個參數值
        t1 = time.time()  # 記錄訓練開始的時間
        score_list = list()  # 建立空列表用於存放不同交叉檢驗下各個評估指標的詳細數據
        print ('set parameters: %s' % parameter)  # 打印當前模型使用的參數
        for score_method in score_methods:  # 循環讀出每個交叉檢驗指標
            model_pipe.set_params(model_adaboost__n_estimators=parameter)  # 通過“管道”設置分類模型參數
            score_tmp = cross_val_score(model_pipe, X, y, scoring=score_method, cv=cv)  # 使用交叉檢驗計算指定指標的得分
            score_list.append(score_tmp)  # 將交叉檢驗得分存儲到列表
        score_matrix = pd.DataFrame(np.array(score_list), index=score_methods)  # 將交叉檢驗詳細數據轉換為矩陣
        score_mean = score_matrix.mean(axis=1).rename('mean')  # 計算每個評估指標的均值
        score_std = score_matrix.std(axis=1).rename('std')  # 計算每個評估指標的標準差
        score_pd = pd.concat([score_matrix, score_mean, score_std], axis=1)  # 將原始詳細數據和均值、標準差合併
        mean_list.append(score_mean)  # 將每個參數得到的各指標均值追加到列表
        std_list.append(score_std)  # 將每個參數得到的各指標標準差追加到列表
        print (score_pd.round(2))  # 打印每個參數得到的交叉檢驗指標數據，只保留2位小數
        print ('-' * 60)
        t2 = time.time()  # 計算每個參數下算法用時
        tt = t2 - t1  # 計算時間間隔
        print ('time: %s' % str(tt))  # 打印時間間隔
    mean_matrix = np.array(mean_list).T  # 建立所有參數得到的交叉檢驗的均值矩陣
    std_matrix = np.array(std_list).T  # 建立所有參數得到的交叉檢驗的標準差矩陣
    mean_pd = pd.DataFrame(mean_matrix, index=score_methods, columns=n_estimators)  # 將均值矩陣轉換為數據框
    std_pd = pd.DataFrame(std_matrix, index=score_methods, columns=n_estimators)  # 將均值標準差轉換為數據框
    print ('Mean values for each parameter:')
    print (mean_pd)  # 打印輸出均值矩陣
    print ('Std values for each parameter:')
    print (std_pd)  # 打印輸出標準差矩陣
    print ('-' * 60)
    return transform

> 資料載入、檢視與前處理

In [9]:
# 數據應用
# 加載數據集
raw_data = pd.read_excel('./data/order.xlsx', sheet_name='Sheet1')  # 讀出Excel的第一個sheet
X = raw_data.drop('response', axis=1)  # 分割X
y = raw_data['response']  # 分割y

- edu_ages: 受教育年數
- user_level: 客戶等級，[1, 7]
- industry: 客戶行業類別，[1, 15]
- value_level: 客戶價值度分級，[1, 6]
- act_level: 客戶活躍度分級，[1, 5]
- blue_money: 優惠藍券使用金額
- red_money: 優惠紅券使用金額

In [10]:
# 數據檢視和預處理
set_summary(raw_data)  # 基本狀態查看
na_summary(raw_data)  # 缺失值檢視(value_level, blue_money, work_hours, response沒有遺缺)
label_summary(raw_data)  # 類樣本均衡均衡檢視
X_t1 = na_replace(X)  # 替換缺失值
X_t2 = type_con(X_t1)  # 數據類型轉換
X_new, enc = symbol_con(X_t2, enc_object=None, train=True)  # 將分類和順序變數轉換為標籤
print(enc)

Data Overview
Records: 39999	Dimension13
------------------------------
    age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0  39.0          77516.0  1.0      13.0         1.0       1.0            1   
1  50.0          83311.0  1.0      13.0         2.0       2.0            2   
2  38.0         215646.0  2.0       9.0         3.0       3.0            1   
3  53.0         234721.0  2.0       7.0         2.0       3.0            2   
4  28.0         338409.0  1.0      13.0         2.0       4.0            3   

   act_level  sex  blue_money  red_money  work_hours  region  response  
0        1.0  1.0        2174        0.0          40     1.0         0  
1        1.0  1.0           0        0.0          13     1.0         0  
2        1.0  1.0           0        0.0          40     1.0         0  
3        2.0  1.0           0        0.0          40     1.0         0  
4        2.0  0.0           0        0.0          40     2.0         0  
---------------------

> 訓練、測試與預測

In [11]:
import warnings
warnings.filterwarnings("ignore")
# 分類模型訓練
transform = get_best_model(X_new, y)  # 獲得最佳分類模型參數信息
transform.fit(X_new, y)  # 應用特征選擇對象選擇要參與建模的特征變量
X_final = transform.transform(X_new)  # 獲得具有顯著性特征的特征變量
final_model = AdaBoostClassifier(n_estimators=100)  # 從打印的參數均值和標準差信息中確定參數並建立分類模型對象
final_model.fit(X_final, y)  # 訓練模型

set parameters: 20
              0     1     2     3     4  mean   std
accuracy   0.85  0.85  0.86  0.86  0.86  0.85  0.01
f1         0.66  0.64  0.66  0.66  0.66  0.66  0.01
precision  0.72  0.74  0.76  0.78  0.77  0.75  0.02
recall     0.60  0.57  0.59  0.58  0.57  0.58  0.01
roc_auc    0.91  0.90  0.91  0.91  0.91  0.91  0.00
------------------------------------------------------------
time: 10.964340925216675
set parameters: 50
              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.86  0.87  0.86  0.86  0.00
f1         0.66  0.66  0.67  0.69  0.68  0.67  0.01
precision  0.75  0.76  0.77  0.78  0.77  0.76  0.01
recall     0.59  0.58  0.59  0.62  0.61  0.60  0.02
roc_auc    0.91  0.91  0.91  0.92  0.92  0.91  0.00
------------------------------------------------------------
time: 22.95265507698059
set parameters: 80
              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.86  0.87  0.86  0.86  0.00
f1         0.67  0.67  0.68  0.70  0.68  0.6

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [12]:
# 新數據集做預測
new_data = pd.read_excel('./data/order.xlsx', sheet_name='Sheet2')  # 讀取要預測的數據集('Sheet2')
final_reponse = new_data['final_response']  # 獲取最終的目標變量值
new_data = new_data.drop('final_response', axis=1)  # 獲得預測的輸入變量X
set_summary(new_data)  # 基本狀態查看
na_summary(new_data)  # 缺失值審查
new_X_t1 = na_replace(new_data)  # 替換缺失值
new_X_t2 = type_con(new_X_t1)  # 數據類型轉換
new_X_t3 = symbol_con(new_X_t2, enc_object=enc, train=False)  # 將分類和順序數據轉換為標籤
new_X_final = transform.transform(new_X_t3)  # 對數據集做特征選擇

Data Overview
Records: 8843	Dimension12
------------------------------
   age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0   61           243019   10         1         2.0       7.0            2   
1   33           215596    4         5         2.0       7.0            2   
2   25            31350    2        10         1.0       5.0            1   
3   23           246965    2        10         1.0      10.0            4   
4   28            99838    1        13         1.0       2.0            1   

   act_level  sex  blue_money  red_money  work_hours  region  
0          1    1           0          0          40     1.0  
1          5    1           0          0          40     6.0  
2          1    1           0          0          40     1.0  
3          2    1           0          0          40     1.0  
4          1    1           0          0          38     5.0  
------------------------------
Data DESC
               age  total_pageviews          ed

In [13]:
# 輸出預測值以及預測概率
predict_labels = pd.DataFrame(final_model.predict(new_X_final), columns=['labels'])  # 獲得預測標籤0或1
predict_labels_pro = pd.DataFrame(final_model.predict_proba(new_X_final), columns=['pro1', 'pro2'])  # 獲得預測概率
predict_pd = pd.concat((new_data, predict_labels, predict_labels_pro), axis=1)  # 將預測標籤、預測數據和原始數據X合並
print ('Predict info')
print (predict_pd.head())  # 打印前2條結果
print ('-' * 60)

Predict info
   age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0   61           243019   10         1         2.0       7.0            2   
1   33           215596    4         5         2.0       7.0            2   
2   25            31350    2        10         1.0       5.0            1   
3   23           246965    2        10         1.0      10.0            4   
4   28            99838    1        13         1.0       2.0            1   

   act_level  sex  blue_money  red_money  work_hours  region  labels  \
0          1    1           0          0          40     1.0       0   
1          5    1           0          0          40     6.0       0   
2          1    1           0          0          40     1.0       0   
3          2    1           0          0          40     1.0       0   
4          1    1           0          0          38     5.0       0   

       pro1      pro2  
0  0.504053  0.495947  
1  0.507486  0.492514  
2  0.511623  0.4883

In [14]:
# 將預測結果寫入Excel
writer = pd.ExcelWriter('./data/order_predict_result.xlsx')  # 創建寫入文件對象
predict_pd.to_excel(writer, 'Sheet1')  # 將數據寫入sheet1
writer.save()  # 保存文件

In [15]:
# 後續--與實際效果的比較
print ('final accuracy: {0}'.format(accuracy_score(final_reponse, predict_labels))) # 0.86

final accuracy: 0.8624901051679295


## 參考文獻 Reference: 

- Raschka, S. (2015), Python Machine Learning: Unlock deeper insights into machine learning with this vital guide to cutting-dege predictive analytics, PACKT Publishing.
- 宋天龍 (2018), Python數據分析與數據化化運營, 機械工業出版社.